<a href="https://colab.research.google.com/github/shanthikp246/OpenGPT/blob/main/finetune_gemma_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Connect to T4 runtime for fine-tuning



In [ ]:
!nvidia-smi

Sun Jul  7 00:20:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Load necessary Pip packages

In [ ]:
!pip install -q datasets trl peft bitsandbytes accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.1 MB/s eta 0:00:00
ERROR: pip's 



*   Create a Quantization config to reduce memory usage.
*   The free tier of google colab only offers 14GB of memory.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



*   Log into HuggingFace
*   Create a HuggingFace account, create api keys with write access and login



In [ ]:
from google.colab import userdata

from huggingface_hub import notebook_login
notebook_login()



*   Gemma-2b model is gated. you have accept T&C before you can use it. Accept it on HuggingFace website before proceeding
*   Pass in the quantization config you created earlier and load the model onto GPU 0



In [ ]:
model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]



*  generate text before fine tuning
*   You can see how fine-tuning changes the output after fine-tuning



In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Quote: Imagination isThe quote is about the power of imagination and how it can help us to see the world in new


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes")
dataset["train"][100]


Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

{'quote': '“I\'m in love with you," he said quietly."Augustus," I said."I am," he said. He was staring at me, and I could see the corners of his eyes crinkling. "I\'m in love with you, and I\'m not in the business of denying myself the simple pleasure of saying true things. I\'m in love with you, and I know that love is just a shout into the void, and that oblivion is inevitable, and that we\'re all doomed and that there will come a day when all our labor has been returned to dust, and I know the sun will swallow the only earth we\'ll ever have, and I am in love with you.”',
 'author': 'John Green,',
 'tags': ['doomed', 'inevitable', 'love', 'oblivion', 'pleasure', 'simple']}



*   Add padding and truncation to handle variable length samples
*   Create a smaller subset of the full training set to reduce time



In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["quote"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
smaller_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
smaller_dataset[100]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'quote': '“Today a reader, tomorrow a leader.”',
 'author': 'Margaret Fuller',
 'tags': ['read', 'reader', 'reading'],
 'input_ids': [2,
  235366,
  15528,
  476,
  11554,
  235269,
  14610,
  476,
  9746,
  1816,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



*   Free google colab provides very little memory. Work around it using PEFT
*   See https://huggingface.co/blog/gemma-peft#why-peft



In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)




*   Declare supervised fine tuning (SFT) trainer and train
*  Define a formatting function on how the model should generate output




In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}<eos>"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        hub_model_id = "shanthi/gemma-2b-lora-ft"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:494: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passe

Step,Training Loss
1,7.511300
2,5.831000
3,6.147100
4,5.966700
5,6.715200
6,6.432400
7,4.857200
8,4.775400
9,5.521500
10,5.402400


TrainOutput(global_step=10, training_loss=5.916016292572022, metrics={'train_runtime': 14.4141, 'train_samples_per_second': 2.775, 'train_steps_per_second': 0.694, 'total_flos': 17112602542080.0, 'train_loss': 5.916016292572022, 'epoch': 0.01594896331738437})



*   Saved the fine tuned model to HuggingFace Hub
*   Go to your HuggingFace account and create a model repo. In my case it is shanthi/gemma-2b-lora-ft
*   Once created we can save the artifacts of the fine tuned model.



In [ ]:
trainer.model.save_pretrained("gemma-2b-lora-ft")
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
trainer.model.push_to_hub("gemma-2b-lora-ft", use_temp_dir=False)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shanthi/gemma-2b-lora-ft/commit/bf60a9e0e26061258a75a284a4018a1122dedc06', commit_message='Upload model', commit_description='', oid='bf60a9e0e26061258a75a284a4018a1122dedc06', pr_url=None, pr_revision=None, pr_num=None)



*   Download the fine-tuned PEFT model and redo inference
*   be sure to use the same base model id you used above in fine tuning.



In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "google/gemma-2b"
adapter_model_name = "shanthi/gemma-2b-lora-ft"

model = AutoModelForCausalLM.from_pretrained(model_id)
model = PeftModel.from_pretrained(model, adapter_model_name)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.to(device).generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more than just a daydream. It's the key to unlocking the future." - Elon Musk


